In [112]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import kss
import re
import networkx as nx
import warnings
warnings.filterwarnings("ignore")

In [113]:
#Mecab 객체 생성
mecab = Mecab()

In [114]:
# df 불러오기
df = pd.read_csv("./test.csv", header=None)
df.columns = ['article']

In [115]:
df = df.iloc[0:5]

In [116]:
# 중복 제거
df = df.drop_duplicates('article')
df.reset_index(inplace=True,drop=True)

In [117]:
#stopwords list화
temp = pd.read_csv("./stop_words.csv",header = None, usecols = [0], names = ['stopwords'])                                                
stopwords = []
for i in range(temp.shape[0]):
    stopwords.append(temp['stopwords'][i])

In [118]:
result = []
for i in range(len(df)):
    removed = re.sub('[-=+,#/\?©:^$©@*\"※~&.·%｜!』\\‘◇”|\(\)\[\]\<\>`\'’“…》]', '', df['article'][i])
    tokenized = mecab.morphs(removed)
    for j in range(len(tokenized)-1,-1,-1):
        if tokenized[j] in stopwords:
            tokenized.remove(tokenized[j])
    result.append(tokenized)
df['token'] = result

In [119]:
df['token'][0]

['영국',
 '옥스퍼드',
 '대',
 '신종',
 '코로나',
 '바이러스',
 '감염증',
 '코로나',
 '19',
 '백신',
 '연구',
 '로이터',
 '뉴스',
 '1',
 '서울',
 '뉴스',
 '1',
 '한상희',
 '기자',
 '전',
 '세계',
 '신종',
 '코로나',
 '바이러스',
 '감염증',
 '코로나',
 '19',
 '백신',
 '연구',
 '중',
 '중',
 '가장',
 '앞선',
 '평가',
 '받',
 '는',
 '영국',
 '옥스퍼드',
 '대',
 '연구',
 '진이',
 '최종',
 '임상',
 '3',
 '상',
 '시험',
 '긍정',
 '적',
 '인',
 '결과',
 '얻',
 '었',
 '다고',
 '1',
 '현지',
 '밝혔',
 '다',
 '로이터',
 '통신',
 '따르',
 '면',
 '사라',
 '길버트',
 '옥스퍼드',
 '대',
 '의대',
 '교수',
 '는',
 '이날',
 '의회',
 '출석',
 '코로나',
 '19',
 '백신',
 '임상',
 '3',
 '상',
 '진행',
 '상황',
 '대한',
 '질문',
 '백신',
 '후보',
 '물질',
 '적절',
 '한',
 '종류',
 '면역',
 '반응',
 '보였',
 '다',
 '고',
 '답',
 '했',
 '다',
 '옥스퍼드',
 '대',
 '는',
 '영국',
 '제약사',
 '아스트라',
 '카',
 '인체',
 '해',
 '없',
 '는',
 '바이러스',
 '코로나',
 '바이러스',
 '유전자',
 '끼워',
 '넣',
 '은',
 '유전자',
 '재조합',
 '백신',
 'AZD',
 '1222',
 '개발',
 '이달',
 '8000',
 '명',
 '임상시험',
 '자원자',
 '대상',
 '3',
 '상',
 '시험',
 '고',
 '있',
 '다',
 '앞서',
 '세계',
 '보건',
 '기구',
 'WHO',
 '는',
 '세계',
 '백신',
 '연구',
 '중',
 '옥스퍼드',
 '아스트

In [121]:
# Word2Vec 모델링
# news_tokens = df['tokenized'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8
model = Word2Vec.load('word2vec.model')

# model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [122]:
#벡터화 작업
def vectorize(tokens):
    result = []
    for token in tokens:
        vectors = []
        for sentence in token:
            init_v = np.array([0.0]*v_dimension)
            count = 0
            for word in sentence:
                word_vectors = model.wv
                if word in word_vectors.vocab:
                    count +=1
                    v = model.wv[word]
                    init_v = init_v + v
            init_v = init_v / count    
            vectors.append(init_v)
        result.append(vectors)
    
    frame = {'article' : df['article'], 'token': df['token'], 'vector': result }
    result = pd.DataFrame(frame) 
    
    return result

In [123]:
df = vectorize(df['token'])

In [124]:
#코사인유사도 매트릭스
def similarity_matrix(vectors):
    sim_mat = np.zeros([len(vectors), len(vectors)])
    for i in tqdm(range(len(vectors))):
        for j in range(len(vectors)):
            sim_mat[i][j] = cosine_similarity(vectors[i].reshape(1, v_dimension), vectors[j].reshape(1, v_dimension))[0,0]
    return sim_mat


In [125]:
df['SimMatrix'] = df['vector'].apply(similarity_matrix)

100%|██████████| 129/129 [00:05<00:00, 24.34it/s]


In [126]:
df

,article,token,vector,SimMatrix
0,영국 옥스퍼드대 신종 코로나바이러스 감염증 코로나19 백신 연구. © 로이터 뉴스1...,"[영국, 옥스퍼드, 대, 신종, 코로나, 바이러스, 감염증, 코로나, 19, 백신,...","[[0.06985935149714351, -0.025657336227595806, ...","[[0.9999999999999998, -0.1314286774940165, 0.2..."
1,크리스탈지노믹스가 수술이 힘든 진행성췌장암 환자에게 아이발티노스타트와 젬시타빈 얼로...,"[크리스탈, 지노, 믹스, 수술, 힘든, 진행, 성, 췌장암, 환자, 발티노스, 타...","[[-0.08433880284428596, -0.8906333353370428, -...","[[0.9999999999999999, -0.09545740665889839, 0...."
2,제주시는 최근 “신산공원 내 저류지를 개방해 반려동물 자율쉼터를 운영겠다”고 했다....,"[제주, 시, 는, 최근, 신산, 공원, 내, 저류지, 개방, 해, 반려, 동물, ...","[[0.27231060713529587, 0.7067987825721502, 0.6...","[[0.9999999999999999, 0.4185591005579885, -0.0..."
3,5월엔 43.1 서울경제 미국 공급관리협회 ISM 는 1일 현지시간 6월 제조업 구...,"[5, 엔, 431, 서울경제, 미국, 공급, 관리, 협회, ISM, 는, 1, 현...","[[1.2809516191482544, -0.4342169165611267, 2.3...","[[1.0000000000000004, 0.12217151691247938, 0.7..."
4,서울경제 극자외선용 EUV 포토레지스트를 생산하는 일본 기업 TOK가 국내에서 생산...,"[서울경제, 극자외선, 용, EUV, 포토레지스트, 생산, 는, 일본, 기업, TO...","[[0.06795791210606694, -0.020151667529717088, ...","[[0.9999999999999999, 0.3334628596227865, -0.0..."


In [107]:
#코사인 유사도를 활용해서 각 문장마다 점수 매기기
#pagerank 사용
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

--------- 이거 이렇게 하면 안될듯... word 하나당 벡터 300개씩 해버리면 계산량이 개지옥임..... 아무래도 keyword extraction은 tf-idf 써야할듯 ㅠㅠ------

참고: https://lovit.github.io/nlp/2019/04/30/textrank/